Tens of thousands of Canadians die annually from heart attacks. Two-thirds of those who suffered these heart attacks, not due to a blood clot, had an undiagnosed heart disease. The failure to diagnose underlying heart disease has led to the high death rates experienced by those with heart attacks. Those with diagnosed heart disease are able to be monitored and helped in the event of heart problems, potentially saving the patient from experiencing a heart attack early. Thus, we have a great need to effectively and accurately evaluate a patient's risk for heart disease. This is why it is important to explore the ability to classify if an individual is at risk of heart disease.

Thus, we ask the question: Is it possible to classify individuals into different levels of heart disease risk (low risk, moderate risk, or high risk) based on blood pressure readings, cholesterol, and other clinical features such as heart rate, ST depression, and thallium stress test results?

To train an algorithm to potentially answer this question, we are using the Cleveland database for heart disease. This database consists of 303 patients who were admitted to the Cleveland Clinic between 1981 and 1984. These patients had no history of heart disease and had various clinical metrics performed on them, as well as detailed documentation of their medical history, lifestyle, and families’ medical history.




In [30]:
# Please uncomment the following cell to install the altair in case your package is not up-to-date

In [31]:
# pip install -U altair

In [32]:
# Please uncomment the following cell to install the package if it is missing

In [33]:
# pip install ucimlrepo

In [36]:
import altair as alt
import pandas as pd
from ucimlrepo import fetch_ucirepo
from sklearn.model_selection import train_test_split

In [37]:
# import dataset
heart_disease_dataset = fetch_ucirepo(name = 'Heart Disease')

# filter dataframe
heart_disease = heart_disease_dataset.data.original
heart_disease.rename(columns = {
                          "fbs" : "fasting_blood_sugar",
                          "chol" : "cholesterol", 
                          "cp":"type_chestpain",
                          "restecg" : "resting_ecg",
                          "thalach" : "max_heart_rate",
                          "exang" : "exercise_induced_angina",
                          "oldpeak" : "ST_depression", 
                          "slope" : "ST_segment_slope", 
                          "ca" : "num_major_vessels", 
                          "thal" : "thallium_stress_test", #not sure
                          "num" : "diagnosis",
                          "trestbps" : "resting_bp"
}, inplace = True)

heart_disease = heart_disease[["cholesterol","type_chestpain","max_heart_rate","resting_bp","diagnosis"]]

heart_disease

,cholesterol,type_chestpain,max_heart_rate,resting_bp,diagnosis
0,233,1,150,145,0
1,286,4,108,160,2
2,229,4,129,120,1
3,250,3,187,130,0
4,204,2,172,130,0
...,...,...,...,...,...
298,264,1,132,110,1
299,193,4,141,144,2
300,131,4,115,130,3
301,236,2,174,130,1


In [38]:
# split data
heart_disease_train, heart_disease_test = train_test_split(heart_disease, train_size = 0.75)

# table including: 
# number of observations in each class, 
# mean, mode, std deviation of the predictor variables,
# and how many rows have missing data

num_observations = len(heart_disease_train.index)
predictor_modes = heart_disease_train.apply("mode").iloc[[0]].melt()

table = pd.DataFrame({
    'variables' : heart_disease_train.columns,
    'missing_values' : heart_disease_train.isnull().sum().values,
    'type' : heart_disease_train.dtypes.values,
    'mode' : predictor_modes["value"], 
    'mean' : heart_disease_train.apply("mean").values,
    'median' : heart_disease_train.apply("median").values,
    'std_deviation' : heart_disease_train.apply("std").values
})

table

,variables,missing_values,type,mode,mean,median,std_deviation
0,cholesterol,0,int64,197,243.876652,239.0,53.560756
1,type_chestpain,0,int64,4,3.127753,3.0,0.993998
2,max_heart_rate,0,int64,163,150.008811,152.0,22.081705
3,resting_bp,0,int64,120,132.537445,130.0,18.357853
4,diagnosis,0,int64,0,0.969163,0.0,1.256460


In [39]:
# To plot distribution of predictor variables
# we need to replace the diagnosis values with their corresponding names
heart_disease_to_plot = heart_disease_train.copy()

# any value larger than 0 will be classified as heart disease
# any value equal to 0 will be classified as no heart disease
heart_disease_to_plot['diagnosis'] = heart_disease_to_plot['diagnosis'].replace([1,2,3,4], "heart disease")
heart_disease_to_plot['diagnosis'] = heart_disease_to_plot['diagnosis'].replace([0], "no heart disease")

heart_disease_to_plot

,cholesterol,type_chestpain,max_heart_rate,resting_bp,diagnosis
272,311,4,120,140,heart disease
15,168,3,174,150,no heart disease
213,228,4,165,178,heart disease
299,193,4,141,144,heart disease
82,321,3,182,140,no heart disease
...,...,...,...,...,...
142,205,2,184,128,no heart disease
11,294,2,153,140,no heart disease
172,249,4,143,174,heart disease
248,212,4,168,125,heart disease


In [40]:
# blood pressure
bp_hist = alt.Chart(heart_disease_to_plot).mark_bar().encode(
    x=alt.X("resting_bp:Q", bin = True).title("Blood Pressure"),
    y=alt.Y("count()").stack(False),
    color="diagnosis:N"
).properties(
    title = "Distribution of Blood Pressure"
)

bp_hist

alt.Chart(...)

In [41]:
# cholesterol
chol_hist = alt.Chart(heart_disease_to_plot).mark_bar().encode(
    x=alt.X("cholesterol:Q", bin = True).title("Cholesterol"),
    y=alt.Y("count()").stack(False),
    color="diagnosis:N"
).properties(
    title = "Distribution of Cholesterol"
)

chol_hist

alt.Chart(...)

In [42]:
# chest pain type
cp_hist = alt.Chart(heart_disease_to_plot).mark_bar().encode(
    x=alt.X("type_chestpain:Q",bin=True).title("Chest Pain Type").scale(alt.Scale(domain = [0,4])),
    y=alt.Y("count()"),
    color="diagnosis:N"
).properties(
    title = "Distribution of Chest Pain Type"
)

cp_hist

alt.Chart(...)

In [43]:
# heart rate
hr_hist = alt.Chart(heart_disease_to_plot).mark_bar().encode(
    x=alt.X("max_heart_rate:Q", bin = True).title("Max Heart Rate"),
    y=alt.Y("count()").stack(False),
    color="diagnosis:N"
).properties(
    title = "Distribution of Heart Rate"
)

hr_hist

alt.Chart(...)